In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load your data
df = pd.read_csv(
    r'C:\Users\Jouke\Documents\evedata-logger\output\market_data_with_names_merged.csv',
    parse_dates=['date']
)
item_names = sorted(df['type_name'].unique())
item_dropdown = widgets.Combobox(
    options=item_names,
    placeholder='Type or select an item',
    description='Item:',
    ensure_option=True,
    continuous_update=False
)
ma_slider = widgets.IntSlider(
    value=7, min=2, max=60, step=1, description='MA Window', continuous_update=False
)
output = widgets.Output()
display(item_dropdown, ma_slider, output)


Combobox(value='', continuous_update=False, description='Item:', ensure_option=True, options=(' Tyrant Blue Sa…

IntSlider(value=7, continuous_update=False, description='MA Window', max=60, min=2)

Output()

In [2]:
def plot_ma_crossover(item, ma_window):
    with output:
        clear_output(wait=True)
        if not item or item not in df['type_name'].values:
            print("Please select a valid item.")
            return
        item_df = df[df['type_name'] == item].sort_values('date').copy()
        item_df['MA'] = item_df['average'].rolling(window=ma_window, min_periods=1).mean()

        # Buy (price crosses above MA), Sell (below MA)
        buy_signals = (item_df['average'] > item_df['MA']) & (item_df['average'].shift(1) <= item_df['MA'].shift(1))
        sell_signals = (item_df['average'] < item_df['MA']) & (item_df['average'].shift(1) >= item_df['MA'].shift(1))

        print(f"MA window: {ma_window} days | Buy signals: {buy_signals.sum()} | Sell signals: {sell_signals.sum()}")
        plt.figure(figsize=(12,6))
        plt.plot(item_df['date'], item_df['average'], label='Price', color='blue')
        plt.plot(item_df['date'], item_df['MA'], label=f'{ma_window}-Day MA', color='orange')
        plt.scatter(item_df.loc[buy_signals, 'date'], item_df.loc[buy_signals, 'average'],
                    label='Buy', marker='^', color='green', s=100)
        plt.scatter(item_df.loc[sell_signals, 'date'], item_df.loc[sell_signals, 'average'],
                    label='Sell', marker='v', color='red', s=100)
        plt.title(f"{item} — Price & {ma_window}-Day MA\nBuy/Sell Signals Shown")
        plt.xlabel("Date")
        plt.ylabel("Average Price (ISK)")
        plt.legend()
        plt.grid(True, linestyle=':')
        plt.tight_layout()
        plt.show()

widgets.interactive_output(
    plot_ma_crossover,
    {'item': item_dropdown, 'ma_window': ma_slider}
)


Output()

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load your data
df = pd.read_csv(
    r'C:\Users\Jouke\Documents\evedata-logger\output\market_data_with_names_merged.csv',
    parse_dates=['date']
)
item_names = sorted(df['type_name'].unique())
item_dropdown = widgets.Combobox(
    options=item_names,
    placeholder='Type or select an item',
    description='Item:',
    ensure_option=True,
    continuous_update=False
)
ma_slider = widgets.IntSlider(
    value=7, min=2, max=60, step=1, description='MA Window', continuous_update=False
)
output = widgets.Output()
display(item_dropdown, ma_slider, output)

def plot_ma_crossover(item, ma_window):
    with output:
        clear_output(wait=True)
        if not item or item not in df['type_name'].values:
            print("Please select a valid item.")
            return
        item_df = df[df['type_name'] == item].sort_values('date').copy()
        item_df['MA'] = item_df['average'].rolling(window=ma_window, min_periods=1).mean()

        # Buy (price crosses above MA), Sell (below MA)
        buy_signals = (item_df['average'] > item_df['MA']) & (item_df['average'].shift(1) <= item_df['MA'].shift(1))
        sell_signals = (item_df['average'] < item_df['MA']) & (item_df['average'].shift(1) >= item_df['MA'].shift(1))

        # Trade simulation
        signals = item_df.copy()
        signals['signal'] = 0
        signals.loc[buy_signals, 'signal'] = 1   # Buy = 1
        signals.loc[sell_signals, 'signal'] = -1 # Sell = -1

        trades = signals[signals['signal'] != 0][['date', 'average', 'signal']].reset_index(drop=True)

        # Make sure first signal is a buy (if not, drop the first sell)
        if not trades.empty and trades.iloc[0]['signal'] == -1:
            trades = trades.iloc[1:].reset_index(drop=True)

        # Only keep pairs of [buy, sell, buy, sell,...]
        if len(trades) % 2 == 1:  # If odd, drop last buy (no sell to close)
            trades = trades.iloc[:-1]

        profits = []
        for i in range(0, len(trades), 2):
            buy_price = trades.iloc[i]['average']
            sell_price = trades.iloc[i+1]['average']
            profits.append(sell_price - buy_price)

        total_profit = sum(profits)
        num_trades = len(profits)

        print(f"MA window: {ma_window} days | Buy signals: {buy_signals.sum()} | Sell signals: {sell_signals.sum()}")
        print(f"Simulated trades: {num_trades}")
        print(f"Total profit (ISK): {total_profit:,.2f}")
        if num_trades:
            print(f"Average profit per trade: {total_profit/num_trades:,.2f}")

        # Plot
        plt.figure(figsize=(12,6))
        plt.plot(item_df['date'], item_df['average'], label='Price', color='blue')
        plt.plot(item_df['date'], item_df['MA'], label=f'{ma_window}-Day MA', color='orange')
        plt.scatter(item_df.loc[buy_signals, 'date'], item_df.loc[buy_signals, 'average'],
                    label='Buy', marker='^', color='green', s=100)
        plt.scatter(item_df.loc[sell_signals, 'date'], item_df.loc[sell_signals, 'average'],
                    label='Sell', marker='v', color='red', s=100)
        plt.title(f"{item} — Price & {ma_window}-Day MA\nBuy/Sell Signals & Profitability")
        plt.xlabel("Date")
        plt.ylabel("Average Price (ISK)")
        plt.legend()
        plt.grid(True, linestyle=':')
        plt.tight_layout()
        plt.show()

widgets.interactive_output(
    plot_ma_crossover,
    {'item': item_dropdown, 'ma_window': ma_slider}
)


Combobox(value='', continuous_update=False, description='Item:', ensure_option=True, options=(' Tyrant Blue Sa…

IntSlider(value=7, continuous_update=False, description='MA Window', max=60, min=2)

Output()

Output()

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np

# Load your data
df = pd.read_csv(
    r'C:\Users\Jouke\Documents\evedata-logger\output\market_history_Ice_Product_2025-07-14_14-46.csv',
    parse_dates=['date']
)

item_names = sorted(df['type_name'].unique())

# 6 comboboxes + checkboxes
combos = []
checks = []
N = 7 
for i in range(N):
    combo = widgets.Combobox(
        options=item_names,
        placeholder=f'Item {i+1}',
        description=f'Item {i+1}:',
        ensure_option=True,
        continuous_update=False
    )
    check = widgets.Checkbox(value=(i==0), description='Show', indent=False)
    combos.append(combo)
    checks.append(check)

ma_slider = widgets.IntSlider(
    value=7, min=2, max=60, step=1,
    description='MA Window',
    continuous_update=False
)
output = widgets.Output()

item_controls = widgets.VBox([widgets.HBox([combo, check]) for combo, check in zip(combos, checks)])
ui = widgets.VBox([item_controls, ma_slider, output])
display(ui)

def profit_for_ma(item_df, ma):
    item_df['MA'] = item_df['average'].rolling(window=ma, min_periods=1).mean()
    buy_signals = (item_df['average'] > item_df['MA']) & (item_df['average'].shift(1) <= item_df['MA'].shift(1))
    sell_signals = (item_df['average'] < item_df['MA']) & (item_df['average'].shift(1) >= item_df['MA'].shift(1))
    signals = item_df.copy()
    signals['signal'] = 0
    signals.loc[buy_signals, 'signal'] = 1
    signals.loc[sell_signals, 'signal'] = -1
    trades = signals[signals['signal'] != 0][['date', 'average', 'signal']].reset_index(drop=True)
    if not trades.empty and trades.iloc[0]['signal'] == -1:
        trades = trades.iloc[1:].reset_index(drop=True)
    if len(trades) % 2 == 1:
        trades = trades.iloc[:-1]
    profits = []
    for i in range(0, len(trades), 2):
        buy_price = trades.iloc[i]['average']
        sell_price = trades.iloc[i+1]['average']
        profits.append(sell_price - buy_price)
    total_profit = sum(profits)
    num_trades = len(profits)
    avg_profit = (total_profit / num_trades) if num_trades else 0
    return total_profit, num_trades, avg_profit

def plot_multi_ma_compare(ma_window, **kwargs):
    with output:
        clear_output(wait=True)
        summary_rows = []
        selected = []
        total_profit = 0
        total_trades = 0
        for i in range(N):
            combo_val = kwargs.get(f'combo{i+1}')
            check_val = kwargs.get(f'check{i+1}')
            if check_val and combo_val and combo_val in df['type_name'].values:
                selected.append(combo_val)
        num_to_plot = len(selected)
        if num_to_plot == 0:
            print("Please select at least one product with 'Show' checked.")
            return
        fig, axes = plt.subplots(num_to_plot, 1, figsize=(13, 5*num_to_plot), sharex=True)
        if isinstance(axes, plt.Axes):
            axes = [axes]
        for idx, item in enumerate(selected):
            item_df = df[df['type_name'] == item].sort_values('date').copy()
            item_profit, item_trades, item_avg = profit_for_ma(item_df.copy(), ma_window)
            axes[idx].plot(item_df['date'], item_df['average'], label='Price', color='blue')
            item_df['MA'] = item_df['average'].rolling(window=ma_window, min_periods=1).mean()
            axes[idx].plot(item_df['date'], item_df['MA'], label=f'{ma_window}-Day MA', color='orange')
            buy_signals = (item_df['average'] > item_df['MA']) & (item_df['average'].shift(1) <= item_df['MA'].shift(1))
            sell_signals = (item_df['average'] < item_df['MA']) & (item_df['average'].shift(1) >= item_df['MA'].shift(1))
            axes[idx].scatter(item_df.loc[buy_signals, 'date'], item_df.loc[buy_signals, 'average'], label='Buy', marker='^', color='green', s=80)
            axes[idx].scatter(item_df.loc[sell_signals, 'date'], item_df.loc[sell_signals, 'average'], label='Sell', marker='v', color='red', s=80)
            axes[idx].set_title(f"{item} | Profit: {item_profit:,.2f} ISK | Trades: {item_trades} | Avg: {item_avg:,.2f}")
            axes[idx].legend()
            axes[idx].grid(True, linestyle=':')
            summary_rows.append([item, f"{item_profit:,.2f}", item_trades, f"{item_avg:,.2f}"])
            total_profit += item_profit
            total_trades += item_trades
        plt.xlabel("Date")
        plt.tight_layout()
        plt.show()

        # Print summary neatly
        print('\n' + '-'*62)
        print(f"{'Item':<30} {'Net Profit (ISK)':>15} {'Trades':>9} {'Avg/Trade':>12}")
        print('-'*62)
        for row in summary_rows:
            print(f"{row[0]:<30} {row[1]:>15} {row[2]:>9} {row[3]:>12}")
        print('-'*62)
        # Show total
        total_avg = (total_profit / total_trades) if total_trades else 0
        print(f"{'TOTAL':<30} {total_profit:>15,.2f} {total_trades:>9} {total_avg:>12,.2f}")
        print('-'*62)

        # Show Top 5 Most Profitable MA's for each selected item
        print("\nTop 5 Most Profitable MA Windows (per item):")
        for item in selected:
            item_df = df[df['type_name'] == item].sort_values('date').copy()
            results = []
            for ma in range(2, 61):
                profit, trades, avg = profit_for_ma(item_df.copy(), ma)
                results.append({'MA': ma, 'Profit': profit, 'Trades': trades, 'Avg': avg})
            results_df = pd.DataFrame(results)
            top5 = results_df.sort_values(by='Profit', ascending=False).head(5)
            print(f"\n{item}:")
            print(f"{'MA':>4} {'Profit':>15} {'Trades':>9} {'Avg/Trade':>12}")
            print('-'*44)
            for _, row in top5.iterrows():
                print(f"{int(row['MA']):>4} {row['Profit']:>15,.2f} {int(row['Trades']):>9} {row['Avg']:>12.2f}")
            print('-'*44)

# Build parameter dict for interactive_output
params = {'ma_window': ma_slider}
for i in range(N):
    params[f'combo{i+1}'] = combos[i]
    params[f'check{i+1}'] = checks[i]

widgets.interactive_output(
    plot_multi_ma_compare,
    params
)


Output()

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load your data
df = pd.read_csv(
    r'C:\Users\Jouke\Documents\evedata-logger\output\market_history_Mineral_2025-07-14_17-10.csv',
    parse_dates=['date']
)
item_names = sorted(df['type_name'].unique())

N = 7
combos = []
checks = []
for i in range(N):
    combo = widgets.Combobox(
        options=item_names,
        placeholder=f'Item {i+1}',
        description=f'Item {i+1}:',
        ensure_option=True,
        continuous_update=False
    )
    check = widgets.Checkbox(value=(i==0), description='Show', indent=False)
    combos.append(combo)
    checks.append(check)

ma_slider = widgets.IntSlider(
    value=7, min=2, max=60, step=1, description='MA Window', continuous_update=False
)
vol_slider = widgets.IntSlider(
    value=14, min=3, max=60, step=1, description='Vol Window', continuous_update=False
)
output = widgets.Output()
widgets_ui = widgets.VBox(
    [widgets.HBox([combos[i], checks[i]]) for i in range(N)] +
    [ma_slider, vol_slider, output]
)
display(widgets_ui)

def plot_multi_ma_vol(
    item0, show0,
    item1, show1,
    item2, show2,
    item3, show3,
    item4, show4,
    item5, show5,
    item6, show6,
    ma_window, vol_window
):
    with output:
        clear_output(wait=True)
        items = [
            (item0, show0),
            (item1, show1),
            (item2, show2),
            (item3, show3),
            (item4, show4),
            (item5, show5),
            (item6, show6)
        ]
        selected = [(item, show) for item, show in items if item and show]
        if not selected:
            print("Select at least one item to plot.")
            return
        fig, ax1 = plt.subplots(figsize=(14, 6))
        prof_summary = []
        total_profit = 0
        total_trades = 0
        total_buy_capital = 0
        top_ma_tables = {}
        for (item, _) in selected:
            item_df = df[df['type_name'] == item].sort_values('date').copy()
            # --- Standard MA/volatility for plotting ---
            item_df['MA'] = item_df['average'].rolling(window=ma_window, min_periods=1).mean()
            item_df['volatility'] = item_df['average'].rolling(window=vol_window, min_periods=1).std()
            buy_signals = (item_df['average'] > item_df['MA']) & (item_df['average'].shift(1) <= item_df['MA'].shift(1))
            sell_signals = (item_df['average'] < item_df['MA']) & (item_df['average'].shift(1) >= item_df['MA'].shift(1))
            signals = item_df.copy()
            signals['signal'] = 0
            signals.loc[buy_signals, 'signal'] = 1
            signals.loc[sell_signals, 'signal'] = -1
            trades = signals[signals['signal'] != 0][['date', 'average', 'signal']].reset_index(drop=True)
            if not trades.empty and trades.iloc[0]['signal'] == -1:
                trades = trades.iloc[1:].reset_index(drop=True)
            if len(trades) % 2 == 1:
                trades = trades.iloc[:-1]
            profits = []
            buy_prices = []
            for i in range(0, len(trades), 2):
                buy_price = trades.iloc[i]['average']
                sell_price = trades.iloc[i+1]['average']
                profits.append(sell_price - buy_price)
                buy_prices.append(buy_price)
            item_profit = sum(profits)
            item_trades = len(profits)
            item_buy_capital = sum(buy_prices)
            item_roi = (item_profit / item_buy_capital * 100) if item_buy_capital > 0 else float('nan')
            prof_summary.append((item, item_profit, item_trades, item_profit/item_trades if item_trades else 0, item_buy_capital, item_roi))
            total_profit += item_profit
            total_trades += item_trades
            total_buy_capital += item_buy_capital
            l, = ax1.plot(item_df['date'], item_df['average'], label=f"{item} Price")
            ax1.plot(item_df['date'], item_df['MA'], linestyle='--', label=f"{item} MA ({ma_window})", color=l.get_color())
            ax1.scatter(item_df.loc[buy_signals, 'date'], item_df.loc[buy_signals, 'average'],
                        marker='^', color=l.get_color(), edgecolors='k', s=80, label=f"{item} Buy")
            ax1.scatter(item_df.loc[sell_signals, 'date'], item_df.loc[sell_signals, 'average'],
                        marker='v', color=l.get_color(), edgecolors='k', s=80, label=f"{item} Sell")
            ax2 = ax1.twinx()
            ax2.plot(item_df['date'], item_df['volatility'], color=l.get_color(), alpha=0.25, linewidth=2, label=f"{item} Volatility")

            # --- Top 10 MA windows by ROI ---
            ma_stats = []
            for ma_w in range(ma_slider.min, ma_slider.max+1):
                test_df = item_df.copy()
                test_df['MA'] = test_df['average'].rolling(window=ma_w, min_periods=1).mean()
                buy_signals = (test_df['average'] > test_df['MA']) & (test_df['average'].shift(1) <= test_df['MA'].shift(1))
                sell_signals = (test_df['average'] < test_df['MA']) & (test_df['average'].shift(1) >= test_df['MA'].shift(1))
                signals = test_df.copy()
                signals['signal'] = 0
                signals.loc[buy_signals, 'signal'] = 1
                signals.loc[sell_signals, 'signal'] = -1
                trades = signals[signals['signal'] != 0][['date', 'average', 'signal']].reset_index(drop=True)
                if not trades.empty and trades.iloc[0]['signal'] == -1:
                    trades = trades.iloc[1:].reset_index(drop=True)
                if len(trades) % 2 == 1:
                    trades = trades.iloc[:-1]
                profits = []
                buy_prices = []
                for i in range(0, len(trades), 2):
                    buy_price = trades.iloc[i]['average']
                    sell_price = trades.iloc[i+1]['average']
                    profits.append(sell_price - buy_price)
                    buy_prices.append(buy_price)
                item_profit = sum(profits)
                item_trades = len(profits)
                item_buy_capital = sum(buy_prices)
                item_roi = (item_profit / item_buy_capital * 100) if item_buy_capital > 0 else float('nan')
                ma_stats.append({
                    "MA": ma_w,
                    "Profit": item_profit,
                    "Trades": item_trades,
                    "Avg/Trade": (item_profit/item_trades if item_trades else 0),
                    "Buy Capital": item_buy_capital,
                    "ROI": item_roi
                })
            ma_stats_df = pd.DataFrame(ma_stats)
            ma_stats_df = ma_stats_df.sort_values('ROI', ascending=False).head(10)
            top_ma_tables[item] = ma_stats_df

        ax1.set_xlabel("Date")
        ax1.set_ylabel("Average Price (ISK)")
        ax1.grid(True, linestyle=':')
        handles1, labels1 = ax1.get_legend_handles_labels()
        handles2, labels2 = ax2.get_legend_handles_labels()
        ax1.legend(handles1 + handles2, labels1 + labels2, loc='upper left', fontsize=10, ncol=2)
        ax2.set_ylabel("Rolling Volatility (Std Dev)")
        plt.title(f"MA/Volatility Analysis — MA: {ma_window} days, Volatility: {vol_window} days")
        plt.tight_layout()
        plt.show()

        # Summary Table
        print("Item".ljust(28), "Net Profit (ISK)".rjust(16), "Trades".rjust(10), "Avg/Trade".rjust(12), "Buy Capital".rjust(14), "ROI (%)".rjust(10))
        print("-"*86)
        for item, profit, trades, avg_trade, buy_cap, roi in prof_summary:
            roi_str = f"{roi:9.2f}%" if not pd.isna(roi) else "   n/a"
            print(f"{item.ljust(28)}{profit:16,.2f}{trades:10}{avg_trade:12.2f}{buy_cap:14,.2f}{roi_str:>10}")
        print("-"*86)
        total_roi = (total_profit / total_buy_capital * 100) if total_buy_capital > 0 else float('nan')
        total_roi_str = f"{total_roi:9.2f}%" if not pd.isna(total_roi) else "   n/a"
        print("TOTAL".ljust(28), f"{total_profit:16,.2f}{total_trades:10}{(total_profit/total_trades if total_trades else 0):12.2f}{total_buy_capital:14,.2f}{total_roi_str:>10}")

        # --- Print Top 10 MAs for each item ---
        print("\nTop 10 MA Windows by ROI (per item):")
        for item, ma_stats_df in top_ma_tables.items():
            print(f"\n{item}:")
            print("  MA      Profit        Trades  Avg/Trade  Buy Capital     ROI (%)")
            print("--------------------------------------------------------------")
            for idx, row in ma_stats_df.iterrows():
                print(f" {int(row['MA']):>3}  {row['Profit']:12,.2f}  {int(row['Trades']):8}  {row['Avg/Trade']:10.2f}  {row['Buy Capital']:12,.2f}  {row['ROI']:9.2f}%")
            print("--------------------------------------------------------------")

widgets.interactive_output(
    plot_multi_ma_vol,
    {
        'item0': combos[0], 'show0': checks[0],
        'item1': combos[1], 'show1': checks[1],
        'item2': combos[2], 'show2': checks[2],
        'item3': combos[3], 'show3': checks[3],
        'item4': combos[4], 'show4': checks[4],
        'item5': combos[5], 'show5': checks[5],
        'item6': combos[6], 'show6': checks[6],
        'ma_window': ma_slider,
        'vol_window': vol_slider
    }
)


Output()

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Configurable
BROKER_FEE = 0.00  # set to 0.03 if you want to add
SALES_TAX = 0.00   # set to 0.02 if you want to add

# Load your data
df = pd.read_csv(r'C:\Users\Jouke\Documents\evedata-logger\output\market_history_Mineral_2025-07-14_17-10.csv', parse_dates=['date'])
item_names = sorted(df['type_name'].unique())

N = 7
combos = []
checks = []
for i in range(N):
    combo = widgets.Combobox(
        options=item_names,
        placeholder=f'Item {i+1}',
        description=f'Item {i+1}:',
        ensure_option=True,
        continuous_update=False
    )
    check = widgets.Checkbox(value=(i==0), description='Show', indent=False)
    combos.append(combo)
    checks.append(check)

ma_slider = widgets.IntSlider(
    value=7, min=2, max=60, step=1, description='MA Window', continuous_update=False
)
vol_slider = widgets.IntSlider(
    value=14, min=3, max=60, step=1, description='Vol Window', continuous_update=False
)
output = widgets.Output()
widgets_ui = widgets.VBox(
    [widgets.HBox([combos[i], checks[i]]) for i in range(N)] +
    [ma_slider, vol_slider, output]
)
display(widgets_ui)

def plot_multi_ma_vol(
    item0, show0,
    item1, show1,
    item2, show2,
    item3, show3,
    item4, show4,
    item5, show5,
    item6, show6,
    ma_window, vol_window
):
    with output:
        clear_output(wait=True)
        items = [
            (item0, show0),
            (item1, show1),
            (item2, show2),
            (item3, show3),
            (item4, show4),
            (item5, show5),
            (item6, show6)
        ]
        selected = [(item, show) for item, show in items if item and show]
        if not selected:
            print("Select at least one item to plot.")
            return
        fig, ax1 = plt.subplots(figsize=(14, 6))
        prof_summary = []
        total_profit = 0
        total_trades = 0
        total_buy_capital = 0
        top_ma_tables = {}
        for (item, _) in selected:
            item_df = df[df['type_name'] == item].sort_values('date').copy()
            item_df['MA'] = item_df['average'].rolling(window=ma_window, min_periods=1).mean()
            item_df['volatility'] = item_df['average'].rolling(window=vol_window, min_periods=1).std()
            # FIX: define buy/sell signals here for plotting
            buy_signals = (item_df['average'] > item_df['MA']) & (item_df['average'].shift(1) <= item_df['MA'].shift(1))
            sell_signals = (item_df['average'] < item_df['MA']) & (item_df['average'].shift(1) >= item_df['MA'].shift(1))

            # --- Volatility Threshold Calculation ---
            vol_thresh = item_df['volatility'].mean() + item_df['volatility'].std()

            # --- MA/Vol for current setting, with volatility stop logic ---
            def simulate_ma_vol_stop(item_df, ma_w, vol_w, vol_thresh):
                item_df = item_df.copy()
                item_df['MA'] = item_df['average'].rolling(window=ma_w, min_periods=1).mean()
                item_df['volatility'] = item_df['average'].rolling(window=vol_w, min_periods=1).std()
                buy_signals = (item_df['average'] > item_df['MA']) & (item_df['average'].shift(1) <= item_df['MA'].shift(1))
                sell_signals = (item_df['average'] < item_df['MA']) & (item_df['average'].shift(1) >= item_df['MA'].shift(1))
                signals = item_df.copy()
                signals['signal'] = 0
                signals.loc[buy_signals, 'signal'] = 1
                signals.loc[sell_signals, 'signal'] = -1
                trades = signals[signals['signal'] != 0][['date', 'average', 'signal', 'volatility']].reset_index(drop=True)
                if not trades.empty and trades.iloc[0]['signal'] == -1:
                    trades = trades.iloc[1:].reset_index(drop=True)
                if len(trades) % 2 == 1:
                    trades = trades.iloc[:-1]
                profits = []
                buy_prices = []
                tdf = item_df.set_index('date')
                for i in range(0, len(trades), 2):
                    buy_row = trades.iloc[i]
                    buy_date = buy_row['date']
                    buy_price = buy_row['average']
                    # Normal planned sell
                    sell_row = trades.iloc[i+1]
                    sell_date = sell_row['date']
                    # --- Check for volatility spike during trade ---
                    held_period = tdf.loc[(tdf.index > buy_date) & (tdf.index <= sell_date)]
                    spike = held_period[held_period['volatility'] >= vol_thresh]
                    if not spike.empty:
                        exit_date = spike.index[0]
                        exit_price = tdf.loc[exit_date]['average']
                        sell_price = exit_price
                        actual_exit = exit_date
                    else:
                        sell_price = sell_row['average']
                        actual_exit = sell_date
                    buy_fee = buy_price * BROKER_FEE
                    sell_fee = sell_price * BROKER_FEE
                    sales_tax = sell_price * SALES_TAX
                    net = (sell_price - buy_price) - (buy_fee + sell_fee + sales_tax)
                    profits.append(net)
                    buy_prices.append(buy_price)
                return profits, buy_prices

            # For display/plot: use selected MA/Vol
            profits, buy_prices = simulate_ma_vol_stop(item_df, ma_window, vol_window, vol_thresh)
            item_profit = sum(profits)
            item_trades = len(profits)
            item_buy_capital = sum(buy_prices)
            item_roi = (item_profit / item_buy_capital * 100) if item_buy_capital > 0 else float('nan')
            prof_summary.append((item, item_profit, item_trades, item_profit/item_trades if item_trades else 0, item_buy_capital, item_roi, vol_thresh))
            total_profit += item_profit
            total_trades += item_trades
            total_buy_capital += item_buy_capital

            # Plotting
            l, = ax1.plot(item_df['date'], item_df['average'], label=f"{item} Price")
            ax1.plot(item_df['date'], item_df['MA'], linestyle='--', label=f"{item} MA ({ma_window})", color=l.get_color())
            ax1.scatter(item_df.loc[buy_signals, 'date'], item_df.loc[buy_signals, 'average'],
                        marker='^', color=l.get_color(), edgecolors='k', s=80, label=f"{item} Buy")
            ax1.scatter(item_df.loc[sell_signals, 'date'], item_df.loc[sell_signals, 'average'],
                        marker='v', color=l.get_color(), edgecolors='k', s=80, label=f"{item} Sell")
            ax2 = ax1.twinx()
            ax2.plot(item_df['date'], item_df['volatility'], color=l.get_color(), alpha=0.25, linewidth=2, label=f"{item} Volatility")
            ax2.axhline(vol_thresh, color=l.get_color(), linestyle=':', alpha=0.5, linewidth=2, label=f"{item} Vol-Exit")

            # --- Top 10 MA windows by ROI, using volatility exit ---
            ma_stats = []
            for ma_w in range(ma_slider.min, ma_slider.max+1):
                profits, buy_prices = simulate_ma_vol_stop(item_df, ma_w, vol_window, vol_thresh)
                profit = sum(profits)
                trades = len(profits)
                buy_cap = sum(buy_prices)
                roi = (profit / buy_cap * 100) if buy_cap > 0 else float('nan')
                ma_stats.append({
                    "MA": ma_w,
                    "Profit": profit,
                    "Trades": trades,
                    "Avg/Trade": (profit/trades if trades else 0),
                    "Buy Capital": buy_cap,
                    "ROI": roi
                })
            ma_stats_df = pd.DataFrame(ma_stats)
            ma_stats_df = ma_stats_df.sort_values('ROI', ascending=False).head(10)
            top_ma_tables[item] = (ma_stats_df, vol_thresh)

        ax1.set_xlabel("Date")
        ax1.set_ylabel("Average Price (ISK)")
        ax1.grid(True, linestyle=':')
        handles1, labels1 = ax1.get_legend_handles_labels()
        handles2, labels2 = ax2.get_legend_handles_labels()
        ax1.legend(handles1 + handles2, labels1 + labels2, loc='upper left', fontsize=10, ncol=2)
        ax2.set_ylabel("Rolling Volatility (Std Dev)")
        plt.title(f"MA/Volatility (with volatility-exit) — MA: {ma_window}d, Vol: {vol_window}d")
        plt.tight_layout()
        plt.show()

        # Summary Table
        print("Item".ljust(28), "Net Profit (ISK)".rjust(16), "Trades".rjust(10), "Avg/Trade".rjust(12), "Buy Capital".rjust(14), "ROI (%)".rjust(10), "Vol Exit".rjust(12))
        print("-"*98)
        for item, profit, trades, avg_trade, buy_cap, roi, vol_thresh in prof_summary:
            roi_str = f"{roi:9.2f}%" if not pd.isna(roi) else "   n/a"
            print(f"{item.ljust(28)}{profit:16,.2f}{trades:10}{avg_trade:12.2f}{buy_cap:14,.2f}{roi_str:>10}{vol_thresh:12.2f}")
        print("-"*98)
        total_roi = (total_profit / total_buy_capital * 100) if total_buy_capital > 0 else float('nan')
        total_roi_str = f"{total_roi:9.2f}%" if not pd.isna(total_roi) else "   n/a"
        print("TOTAL".ljust(28), f"{total_profit:16,.2f}{total_trades:10}{(total_profit/total_trades if total_trades else 0):12.2f}{total_buy_capital:14,.2f}{total_roi_str:>10}")

        # --- Print Top 10 MAs for each item ---
        print("\nTop 10 MA Windows by ROI (per item, with volatility exit):")
        for item, (ma_stats_df, vol_thresh) in top_ma_tables.items():
            print(f"\n{item}: (Volatility Exit Threshold: {vol_thresh:.2f})")
            print("  MA      Profit        Trades  Avg/Trade  Buy Capital     ROI (%)")
            print("--------------------------------------------------------------")
            for idx, row in ma_stats_df.iterrows():
                print(f" {int(row['MA']):>3}  {row['Profit']:12,.2f}  {int(row['Trades']):8}  {row['Avg/Trade']:10.2f}  {row['Buy Capital']:12,.2f}  {row['ROI']:9.2f}%")
            print("--------------------------------------------------------------")

widgets.interactive_output(
    plot_multi_ma_vol,
    {
        'item0': combos[0], 'show0': checks[0],
        'item1': combos[1], 'show1': checks[1],
        'item2': combos[2], 'show2': checks[2],
        'item3': combos[3], 'show3': checks[3],
        'item4': combos[4], 'show4': checks[4],
        'item5': combos[5], 'show5': checks[5],
        'item6': combos[6], 'show6': checks[6],
        'ma_window': ma_slider,
        'vol_window': vol_slider
    }
)


Output()

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# --------- CONFIG ---------
BROKER_FEE = 0.00
SALES_TAX = 0.00

# Load your data
df = pd.read_csv(r'C:\Users\Jouke\Documents\evedata-logger\output\market_history_Mineral_2025-07-14_17-10.csv', parse_dates=['date'])
item_names = sorted(df['type_name'].unique())

# ------- Helper: Simulate MA/vol stop -------
def simulate_ma_vol_stop(item_df, ma_w, vol_w, vol_thresh):
    item_df = item_df.copy()
    item_df['MA'] = item_df['average'].rolling(window=ma_w, min_periods=1).mean()
    item_df['volatility'] = item_df['average'].rolling(window=vol_w, min_periods=1).std()
    buy_signals = (item_df['average'] > item_df['MA']) & (item_df['average'].shift(1) <= item_df['MA'].shift(1))
    sell_signals = (item_df['average'] < item_df['MA']) & (item_df['average'].shift(1) >= item_df['MA'].shift(1))
    signals = item_df.copy()
    signals['signal'] = 0
    signals.loc[buy_signals, 'signal'] = 1
    signals.loc[sell_signals, 'signal'] = -1
    trades = signals[signals['signal'] != 0][['date', 'average', 'signal', 'volatility']].reset_index(drop=True)
    if not trades.empty and trades.iloc[0]['signal'] == -1:
        trades = trades.iloc[1:].reset_index(drop=True)
    if len(trades) % 2 == 1:
        trades = trades.iloc[:-1]
    profits = []
    buy_prices = []
    tdf = item_df.set_index('date')
    for i in range(0, len(trades), 2):
        buy_row = trades.iloc[i]
        buy_date = buy_row['date']
        buy_price = buy_row['average']
        sell_row = trades.iloc[i+1]
        sell_date = sell_row['date']
        # Volatility-exit logic
        held_period = tdf.loc[(tdf.index > buy_date) & (tdf.index <= sell_date)]
        spike = held_period[held_period['volatility'] >= vol_thresh]
        if not spike.empty:
            exit_date = spike.index[0]
            sell_price = tdf.loc[exit_date]['average']
        else:
            sell_price = sell_row['average']
        buy_fee = buy_price * BROKER_FEE
        sell_fee = sell_price * BROKER_FEE
        sales_tax = sell_price * SALES_TAX
        net = (sell_price - buy_price) - (buy_fee + sell_fee + sales_tax)
        profits.append(net)
        buy_prices.append(buy_price)
    return profits, buy_prices

# ------- Widgets/UI -------
item_dropdown = widgets.Combobox(
    options=item_names, placeholder='Type or select an item', description='Item:',
    ensure_option=True, continuous_update=False
)
vol_slider = widgets.IntSlider(
    value=14, min=3, max=60, step=1, description='Vol Window', continuous_update=False
)
split_slider = widgets.FloatSlider(
    value=0.6, min=0.2, max=0.8, step=0.05, description='Train Split', readout_format='.2f'
)
output = widgets.Output()
display(item_dropdown, vol_slider, split_slider, output)

# ----------- Main Plot Function -----------
def plot_roi_overfit(item, vol_window, train_split):
    with output:
        clear_output(wait=True)
        if not item or item not in df['type_name'].values:
            print("Please select a valid item.")
            return
        item_df = df[df['type_name'] == item].sort_values('date').copy()
        n = len(item_df)
        split_idx = int(n * train_split)
        train_df = item_df.iloc[:split_idx].copy()
        test_df = item_df.iloc[split_idx:].copy()
        # Calculate volatility threshold on train period
        train_df['volatility'] = train_df['average'].rolling(window=vol_window, min_periods=1).std()
        vol_thresh = train_df['volatility'].mean() + train_df['volatility'].std()

        rois_train, rois_test = [], []
        mas = range(2, 61)
        for ma_w in mas:
            # Train
            train_profits, train_buys = simulate_ma_vol_stop(train_df, ma_w, vol_window, vol_thresh)
            train_profit = sum(train_profits)
            train_cap = sum(train_buys)
            train_roi = (train_profit / train_cap * 100) if train_cap > 0 else float('nan')
            rois_train.append(train_roi)
            # Test (out-of-sample), keep using train vol_thresh!
            test_profits, test_buys = simulate_ma_vol_stop(test_df, ma_w, vol_window, vol_thresh)
            test_profit = sum(test_profits)
            test_cap = sum(test_buys)
            test_roi = (test_profit / test_cap * 100) if test_cap > 0 else float('nan')
            rois_test.append(test_roi)

        plt.figure(figsize=(12, 6))
        plt.plot(mas, rois_train, label='Train ROI', marker='o')
        plt.plot(mas, rois_test, label='Test ROI (Out-of-sample)', marker='x')
        plt.axhline(0, color='red', linestyle='--')
        plt.axhline(5, color='green', linestyle=':', label='Fee Break-Even')
        plt.title(f"ROI vs MA Window for {item}\nTrain: 0–{split_idx} | Test: {split_idx}–{n}")
        plt.xlabel("MA Window")
        plt.ylabel("ROI (%)")
        plt.legend()
        plt.grid(True, linestyle=':')
        plt.tight_layout()
        plt.show()

        # Show best MA in train/test
        best_ma_train = mas[pd.Series(rois_train).idxmax()] if len(rois_train) > 0 else None
        best_ma_test = mas[pd.Series(rois_test).idxmax()] if len(rois_test) > 0 else None
        print(f"Train period best MA: {best_ma_train} (ROI: {max(rois_train):.2f}%)")
        print(f"Test  period best MA: {best_ma_test} (ROI: {max(rois_test):.2f}%)")
        print("If the test ROI is much lower, your parameter may be overfit!")

widgets.interactive_output(
    plot_roi_overfit,
    {'item': item_dropdown, 'vol_window': vol_slider, 'train_split': split_slider}
)


Combobox(value='', continuous_update=False, description='Item:', ensure_option=True, options=('Chromodynamic T…

IntSlider(value=14, continuous_update=False, description='Vol Window', max=60, min=3)

FloatSlider(value=0.6, description='Train Split', max=0.8, min=0.2, step=0.05)

Output()

Output()